In [ ]:
import numpy as np
import blosc2
import time
import plotly.express as px
import pandas as pd

from blosc2 import NDArray
from typing import Any

import builtins

In [ ]:
def new_permute_dims(arr: NDArray, axes: tuple[int] | list[int] | None = None, **kwargs: Any) -> NDArray:
    if np.isscalar(arr) or arr.ndim < 2:
        return arr

    ndim = arr.ndim
    if axes is None:
        axes = tuple(range(ndim))[::-1]
    else:
        axes = tuple(axis if axis >= 0 else ndim + axis for axis in axes)
        if sorted(axes) != list(range(ndim)):
            raise ValueError(f"axes {axes} is not a valid permutation of {ndim} dimensions")

    new_shape = tuple(arr.shape[axis] for axis in axes)
    if "chunks" not in kwargs or kwargs["chunks"] is None:
        kwargs["chunks"] = tuple(arr.chunks[axis] for axis in axes)

    result = blosc2.empty(shape=new_shape, dtype=arr.dtype, **kwargs)

    # Precomputar info por dimensión
    chunks = arr.chunks
    shape = arr.shape

    for info in arr.iterchunks_info():
        coords = info.coords
        start_stop = [
            (coord * chunk, builtins.min(chunk * (coord + 1), dim))
            for coord, chunk, dim in zip(coords, chunks, shape)
        ]

        src_slice = tuple(slice(start, stop) for start, stop in start_stop)
        dst_slice = tuple(slice(start_stop[ax][0], start_stop[ax][1]) for ax in axes)

        transposed = np.transpose(arr[src_slice], axes=axes)
        result[dst_slice] = np.ascontiguousarray(transposed)

    return result

In [ ]:
def validate_results(result_orig, result_new, shape):
    if not np.allclose(result_orig[:], result_new[:]):
        raise ValueError(f"Mismatch found for shape {shape}")

shapes = [
    (100, 100), (2000, 2000), (3000, 3000), (4000, 4000), (3000, 7000),
    (5000, 5000), (6000, 6000), (7000, 7000), (8000, 8000), (6000, 12000),
    (9000, 9000), (10000, 10000), (10500, 10500), (11000, 11000), (11500, 11500),
    (12000, 12000), (12500, 12500), (13000, 13000), (13500, 13500), (14000, 14000),
    (14500, 14500), (15000, 15000), (16000, 16000), (16500, 16500), (17000, 17000),
    (17500, 17500), (18000, 18000)
]

sizes = []
time_total = []
chunk_labels = []

def numpy_permute(arr: np.ndarray, axes: tuple[int] | list[int] | None = None) -> np.ndarray:
    if axes is None:
        axes = range(arr.ndim)[::-1]
    return np.transpose(arr, axes=axes).copy()

for shape in shapes:
    size_mb = (np.prod(shape) * 8) / (2 ** 20)

    # NumPy transpose
    matrix_numpy = np.linspace(0, 1, np.prod(shape)).reshape(shape)
    t0 = time.perf_counter()
    result_numpy = numpy_permute(matrix_numpy)
    t1 = time.perf_counter()
    time_total.append(t1 - t0)
    sizes.append(size_mb)
    chunk_labels.append("numpy.transpose()")

    # New permute dims (optimized)
    matrix_blosc2 = blosc2.linspace(0, 1, np.prod(shape), shape=shape)
    t0 = time.perf_counter()
    result_new_perm = new_permute_dims(matrix_blosc2)
    t1 = time.perf_counter()
    time_total.append(t1 - t0)
    sizes.append(size_mb)
    chunk_labels.append("blosc2.permute_dims()")

    try:
        validate_results(result_new_perm, result_numpy, shape)
    except ValueError as e:
        print(e)

    print(f"Shape={shape}, Chunk={matrix_blosc2.chunks}: permute_dims={time_total[-2]:.6f}s, numpy={time_total[-1]:.6f}s")

In [ ]:
df = pd.DataFrame({
    "Matrix Size (MB)": sizes,
    "Time (s)": time_total,
    "Implementation": chunk_labels
})

fig = px.line(df,
              x="Matrix Size (MB)",
              y="Time (s)",
              color="Implementation",
              title="Performance: NumPy vs Blosc2",
              width=1000, height=600,
              markers=True)
fig.show()

In [ ]:
%%time
shapes = [
    (100, 100), (1000, 1000), (2000, 2000), (3000, 3000), (4000, 4000),
    (5000, 5000), (6000, 6000), (7000, 7000), (8000, 8000), (9000, 9000),
    (9500, 9500), (10000, 10000), (10500, 10500), (11000, 11000), (11500, 11500),
    (12000, 12000), (12500, 12500), (13000, 13000), (13500, 13500), (14000, 14000),
    (14500, 14500), (15000, 15000), (16000, 16000), (16500, 16500), (17000, 17000)
]

chunkshapes = [None, (150, 300), (1000, 1000), (4000, 4000)]

sizes = []
time_total = []
chunk_labels = []

for shape in shapes:
    size_mb = (np.prod(shape) * 8) / (2 ** 20)

    matrix_np = np.linspace(0, 1, np.prod(shape)).reshape(shape)

    t0 = time.perf_counter()
    result_numpy = np.transpose(matrix_np).copy()
    numpy_time = time.perf_counter() - t0

    time_total.append(numpy_time)
    sizes.append(size_mb)
    chunk_labels.append("NumPy")

    print(f"NumPy:  Shape={shape}, Time = {numpy_time:.6f} s")

    for chunk in chunkshapes:
        matrix_blosc2 = blosc2.asarray(matrix_np)
        matrix_blosc2 = blosc2.linspace(0, 1, np.prod(shape), shape=shape)

        t0 = time.perf_counter()
        result_blosc2 = new_permute_dims(matrix_blosc2, chunks=chunk)
        blosc2_time = time.perf_counter() - t0

        sizes.append(size_mb)
        time_total.append(blosc2_time)
        chunk_labels.append(f"{chunk[0]}x{chunk[1]}" if chunk else "Auto")

        print(f"Blosc2: Shape={shape}, Chunks = {result_blosc2.chunks}, Time = {blosc2_time:.6f} s")

In [ ]:
df = pd.DataFrame({
    "Matrix Size (MB)": sizes,
    "Time (s)": time_total,
    "Chunk Shape": chunk_labels
})

fig = px.line(df,
              x="Matrix Size (MB)",
              y="Time (s)",
              color="Chunk Shape",
              title="Performance of Matrix Transposition (Blosc2 vs NumPy)",
              labels={"value": "Time (s)", "variable": "Metric"},
              width=1000, height=600,
              markers=True)
fig.show()